In [43]:
# Set autoreload
%load_ext autoreload
%autoreload 2

import os
import sys
import pandas as pd
sys.path.append('../app/src')
from config import config

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Create sql database


In [48]:
from langchain_community.utilities import SQLDatabase
db = SQLDatabase.from_uri('sqlite:///../data/artifacts/sql_database.db')
print(db.dialect)
print(db.get_usable_table_names())

sqlite
['financial_advisor_clients']


In [66]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [67]:
agent_executor.invoke(
    "List the total number of clients within each analyst rating position"
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


financial_advisor_clients
Invoking: `sql_db_schema` with `{'table_names': 'financial_advisor_clients'}`



CREATE TABLE financial_advisor_clients (
	"Client" TEXT, 
	"Symbol" TEXT, 
	"Name" TEXT, 
	"Sector" TEXT, 
	"Quantity" REAL, 
	"Buy Price" REAL, 
	"Current Price" REAL, 
	"Market Value" REAL, 
	"Purchase Date" TEXT, 
	"Dividend Yield" REAL, 
	"P/E Ratio" REAL, 
	"52-Week High" REAL, 
	"52-Week Low" REAL, 
	"Analyst Rating" TEXT, 
	"Target Price" REAL, 
	"Risk Level" TEXT, 
	"Client_validation" INTEGER, 
	"Client_Id" TEXT
)

/*
3 rows from financial_advisor_clients table:
Client	Symbol	Name	Sector	Quantity	Buy Price	Current Price	Market Value	Purchase Date	Dividend Yield	P/E Ratio	52-Week High	52-Week Low	Analyst Rating	Target Price	Risk Level	Client_validation	Client_Id
Client_1	DIA	SPDR Dow Jones Industrial Average ETF	ETF	135.0	264.89	574.42	77546.7	12/11/21	1.61	19.2	674.71	493.6	Hold	660.2

{'input': 'List the total number of clients within each analyst rating position',
 'output': 'Here is the total number of clients within each analyst rating position:\n\n- Buy: 50 clients\n- Hold: 50 clients\n- Sell: 50 clients\n- No rating: 5 clients'}

## Adding Few-Shot Prompt

In [68]:
examples = [
    {"input": 'Get the final earning for clients whose Analyst Rating was to sell the shares',
    "query": '''
    SELECT
        fac.Client_Id,
        fac.Client,
        SUM(ROUND(fac.Quantity * (fac."Current Price"-fac."Buy Price"))) AS FinalEarning
    FROM financial_advisor_clients AS fac
    WHERE
        fac."Analyst Rating" = 'Sell'
    GROUP BY
        fac.Client_Id,
        fac.Client
    ORDER BY
        fac.Client_Id ASC
    '''}
]

In [69]:
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    FAISS,
    k=5,
    input_keys=["input"],
)

In [70]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotPromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
    SystemMessagePromptTemplate,
)

system_prefix = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't know" as the answer.

Here are some examples of user inputs and their corresponding SQL queries:"""

few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=PromptTemplate.from_template(
        "User input: {input}\nSQL query: {query}"
    ),
    input_variables=["input", "dialect", "top_k"],
    prefix=system_prefix,
    suffix="",
)

In [71]:
full_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=few_shot_prompt),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

In [72]:
# Example formatted prompt
prompt_val = full_prompt.invoke(
    {
        "input": "How many arists are there",
        "top_k": 5,
        "dialect": "SQLite",
        "agent_scratchpad": [],
    }
)
print(prompt_val.to_string())

System: You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't 

In [73]:
agent = create_sql_agent(
    llm=llm,
    db=db,
    prompt=full_prompt,
    verbose=True,
    agent_type="openai-tools",
)

In [74]:
agent.invoke({"input": "how much the Client_1 got as earnings?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


financial_advisor_clients
Invoking: `sql_db_schema` with `{'table_names': 'financial_advisor_clients'}`



CREATE TABLE financial_advisor_clients (
	"Client" TEXT, 
	"Symbol" TEXT, 
	"Name" TEXT, 
	"Sector" TEXT, 
	"Quantity" REAL, 
	"Buy Price" REAL, 
	"Current Price" REAL, 
	"Market Value" REAL, 
	"Purchase Date" TEXT, 
	"Dividend Yield" REAL, 
	"P/E Ratio" REAL, 
	"52-Week High" REAL, 
	"52-Week Low" REAL, 
	"Analyst Rating" TEXT, 
	"Target Price" REAL, 
	"Risk Level" TEXT, 
	"Client_validation" INTEGER, 
	"Client_Id" TEXT
)

/*
3 rows from financial_advisor_clients table:
Client	Symbol	Name	Sector	Quantity	Buy Price	Current Price	Market Value	Purchase Date	Dividend Yield	P/E Ratio	52-Week High	52-Week Low	Analyst Rating	Target Price	Risk Level	Client_validation	Client_Id
Client_1	DIA	SPDR Dow Jones Industrial Average ETF	ETF	135.0	264.89	574.42	77546.7	12/11/21	1.61	19.2	674.71	493.6	Hold	660.2

{'input': 'how much the Client_1 got as earnings?',
 'output': 'Client_1 earned a total of 132,002.0.'}

In [75]:
agent.invoke({"input": "Hi! my name is Victor. How can you help me?"})



> Entering new SQL Agent Executor chain...
Hi Victor! I can help you interact with a SQL database to retrieve information or answer questions based on the data stored in it. You can ask me to:

- Retrieve specific data or records.
- Summarize or analyze data.
- Provide insights based on certain criteria.
- Answer questions about the structure of the database.

For example, you can ask me things like:
- "Show me the top 10 products by sales."
- "What is the average salary of employees in the marketing department?"
- "List all clients who have made a purchase in the last month."

Just let me know what you need, and I'll do my best to assist you!

> Finished chain.


{'input': 'Hi! my name is Victor. How can you help me?',
 'output': 'Hi Victor! I can help you interact with a SQL database to retrieve information or answer questions based on the data stored in it. You can ask me to:\n\n- Retrieve specific data or records.\n- Summarize or analyze data.\n- Provide insights based on certain criteria.\n- Answer questions about the structure of the database.\n\nFor example, you can ask me things like:\n- "Show me the top 10 products by sales."\n- "What is the average salary of employees in the marketing department?"\n- "List all clients who have made a purchase in the last month."\n\nJust let me know what you need, and I\'ll do my best to assist you!'}

## Adding memory

In [76]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [77]:
message_history = ChatMessageHistory()

In [78]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [85]:
agent_with_chat_history.invoke(
    {"input": "how much the Client_1 got as earnings?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


financial_advisor_clients
Invoking: `sql_db_schema` with `{'table_names': 'financial_advisor_clients'}`



CREATE TABLE financial_advisor_clients (
	"Client" TEXT, 
	"Symbol" TEXT, 
	"Name" TEXT, 
	"Sector" TEXT, 
	"Quantity" REAL, 
	"Buy Price" REAL, 
	"Current Price" REAL, 
	"Market Value" REAL, 
	"Purchase Date" TEXT, 
	"Dividend Yield" REAL, 
	"P/E Ratio" REAL, 
	"52-Week High" REAL, 
	"52-Week Low" REAL, 
	"Analyst Rating" TEXT, 
	"Target Price" REAL, 
	"Risk Level" TEXT, 
	"Client_validation" INTEGER, 
	"Client_Id" TEXT
)

/*
3 rows from financial_advisor_clients table:
Client	Symbol	Name	Sector	Quantity	Buy Price	Current Price	Market Value	Purchase Date	Dividend Yield	P/E Ratio	52-Week High	52-Week Low	Analyst Rating	Target Price	Risk Level	Client_validation	Client_Id
Client_1	DIA	SPDR Dow Jones Industrial Average ETF	ETF	135.0	264.89	574.42	77546.7	12/11/21	1.61	19.2	674.71	493.6	Hold	660.2

{'input': 'how much the Client_1 got as earnings?',
 'chat_history': [HumanMessage(content="hi! I'm bob"),
  AIMessage(content='Hello Bob! How can I assist you today?'),
  HumanMessage(content="what's my name?"),
  AIMessage(content="I don't know.")],
 'output': 'Client_1 earned a total of 132,002.0.'}

In [86]:
agent_with_chat_history.invoke(
    {"input": "what about Client_2?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


financial_advisor_clients
Invoking: `sql_db_schema` with `{'table_names': 'financial_advisor_clients'}`



CREATE TABLE financial_advisor_clients (
	"Client" TEXT, 
	"Symbol" TEXT, 
	"Name" TEXT, 
	"Sector" TEXT, 
	"Quantity" REAL, 
	"Buy Price" REAL, 
	"Current Price" REAL, 
	"Market Value" REAL, 
	"Purchase Date" TEXT, 
	"Dividend Yield" REAL, 
	"P/E Ratio" REAL, 
	"52-Week High" REAL, 
	"52-Week Low" REAL, 
	"Analyst Rating" TEXT, 
	"Target Price" REAL, 
	"Risk Level" TEXT, 
	"Client_validation" INTEGER, 
	"Client_Id" TEXT
)

/*
3 rows from financial_advisor_clients table:
Client	Symbol	Name	Sector	Quantity	Buy Price	Current Price	Market Value	Purchase Date	Dividend Yield	P/E Ratio	52-Week High	52-Week Low	Analyst Rating	Target Price	Risk Level	Client_validation	Client_Id
Client_1	DIA	SPDR Dow Jones Industrial Average ETF	ETF	135.0	264.89	574.42	77546.7	12/11/21	1.61	19.2	674.71	493.6	Hold	660.2

{'input': 'what about Client_2?',
 'chat_history': [HumanMessage(content="hi! I'm bob"),
  AIMessage(content='Hello Bob! How can I assist you today?'),
  HumanMessage(content="what's my name?"),
  AIMessage(content="I don't know."),
  HumanMessage(content='how much the Client_1 got as earnings?'),
  AIMessage(content='Client_1 earned a total of 132,002.0.')],
 'output': 'Here are the details for Client_2:\n\n1. **Meta Platforms Inc. (FB)**\n   - Sector: Communication Services\n   - Quantity: 116\n   - Buy Price: 1346.21\n   - Current Price: 362.21\n   - Market Value: 42016.36\n   - Purchase Date: 3/11/21\n   - Dividend Yield: 2.8%\n   - P/E Ratio: 23.89\n   - 52-Week High: 403.27\n   - 52-Week Low: 240.16\n   - Analyst Rating: Buy\n   - Target Price: 370.07\n   - Risk Level: Low\n\n2. **Tesla Inc. (TSLA)**\n   - Sector: Consumer Discretionary\n   - Quantity: 148\n   - Buy Price: 70.38\n   - Current Price: 71.81\n   - Market Value: 10627.88\n   - Purchase Date: 11/20/21\n   - Dividend Y